In [20]:
%cd -q ../..

In [24]:
import json
import re
import time
from itertools import chain
from pathlib import Path

In [ ]:
with open("data/scdb/missing_by_docket_number.json", "r") as f:
    missing_by_docket_number = json.load(f)

In [35]:
bad_split_filepaths = []
for opinion_filepath in sorted(Path("data/cap/known_authors").glob("*/*.json"), key=lambda x: x.stem):
    docket_number = opinion_filepath.stem
    if docket_number in missing_by_docket_number:
        if len(missing_by_docket_number[docket_number].get("not_in_cap", [])) > 0:
            bad_split_filepaths.append(opinion_filepath)

In [36]:
missing_by_docket_number["00-511"]

{'not_in_cap': ['souter']}

In [ ]:
sum([len(x.get("not_in_scdb", [])) for x in missing_by_docket_number.values()])
len([x for x in missing_by_docket_number.values() if x.get("not_in_cap", None) is not None])
sum([len(x.get("not_in_cap", [])) for x in missing_by_docket_number.values()])

In [ ]:
bad_split_filepaths

In [90]:
re.match("(?=.*justice)((?=.*concurring)|(?=.*dissenting))", "no  dissenting")

In [170]:
def get_heuristic_match(opinion_paragraphs: list[str]):
    for i, paragraph in enumerate(opinion_paragraphs[5:]):
        if re.match("(?=.*justice)((?=.*concurring)|(?=.*dissenting))", paragraph.lower()) is not None:
            if len(paragraph) > 200:
                continue
            #if paragraph.lower().startswith("justice"):
            return opinion_paragraphs[5+i-1:5+i+2]
    return None

In [214]:
def get_heuristic_matches(opinion_paragraphs: list[str]):
    matching_paragraphs = []
    for paragraph in opinion_paragraphs[5:]:
        if re.match("(?=.*justice)((?=.*concurring)|(?=.*dissenting))", paragraph.lower()) is not None:
            # if len(paragraph) > 400:
            #     continue
            # if not (paragraph.lower().startswith("justice") or paragraph.lower().startswith("chief")):
            #     continue
            # if not paragraph.lower().startswith("justice"):
            #     continue
            matching_paragraphs.append(paragraph)
    return matching_paragraphs

def opinion_has_fix(opinion_filepath: Path):
    with open(opinion_filepath, "r") as f:
        text = json.load(f)["text"]
        paragraphs = text.split("\n")
        result = get_heuristic_matches(paragraphs)
        return len(result) > 0, result

In [ ]:
opinion_has_fix(Path("data/cap/known_authors/scalia/05-352.json"))

(True,
 ['It is so ordered.',
  'Justice Alito, with whom The Chief Justice, Justice Kennedy, and Justice Thomas join, dissenting.',
  'I disagree with the Court’s conclusion that a criminal conviction must .automatically be reversed whenever a trial court errs in applying its rules regarding pro hac vice admissions and as a result prevents a defendant from being represented at trial by the defendant’s first-choice attorney. Instead, a defendant should be required to make at least some showing that the trial court’s erroneous ruling adversely affected the quality of assistance that the defendant received. In my view, the majority’s contrary holding is based on an incorrect interpretation of the Sixth Amendment and a misapplication of harmless-error principles. I respectfully dissent.'])

In [216]:
total_matches = 0
correctable_filepaths = []
all_paragraphs = []
for filepath in bad_split_filepaths:
    has_matches, matches = opinion_has_fix(filepath)
    if has_matches:
        correctable_filepaths.append(filepath)
        total_matches += len(matches)
        all_paragraphs.append(matches)
total_matches

1082

In [218]:
all_paragraphs = list(chain(*all_paragraphs))

In [225]:
with open("heuristic_matches.txt", "w") as f:
    f.writelines("\n".join(all_paragraphs))

In [172]:
len(correctable_filepaths)

335

In [ ]:
from IPython.display import clear_output

In [164]:
def check_results(filepaths):
    num_correct = 0
    total = 0
    for filepath in filepaths:
        result = opinion_has_fix(filepath)[1][1]
        print(result, flush=True)
        time.sleep(.1)
        x = input()
        if x == "":
            num_correct += 1
        total += 1
        if x == "q":
            return num_correct, total

        clear_output(wait=True)

In [165]:
correct, total = check_results(correctable_filepaths)

Justice THOMAS, with whom Justice GINSBURGjoins, dissenting.


In [166]:
print(correct, total, correct/total)

106 126 0.8412698412698413


In [ ]:
re.match()

In [19]:
val = "Justice dissenting"
re.match("(?=.*justice)(?=.*concurring)|(?=.*dissenting)", val.lower())

<re.Match object; span=(0, 0), match=''>